<a href="https://colab.research.google.com/github/rmbayer/ASL_NN/blob/master/asl_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN TensorFlow Framework for American Sign Language Detection
### **Bob Bayer and Jonathan Rice 2020**

**Retrieve Datasets**

In [2]:
# Import and mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
# Check drive contents
!ls "/content/drive/My Drive/Sign Language"

 asl-alphabet.zip		      'Sign Language Project Research.gdoc'
'ASL Recognition with Deep Learning'


**The following python code will use the OS library to use Operating System libraries, giving you access to the file system, and the zipfile library allowing you to unzip the data.**

In [0]:
import os
import zipfile

local_zip = '/content/drive/My Drive/Sign Language/asl-alphabet.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
# Extract to a local file and close the connection to the drive file
zip_ref.extractall('/tmp/') 
zip_ref.close()

**Set the directories for the training and test datasets**

In [0]:
train_dir = os.path.join('/tmp/asl_alphabet_train/asl_alphabet_train')
test_dir= os.path.join('/tmp/asl_alphabet_test/asl_alphabet_test')

**Veryify the subfolders of the directory**

In [16]:
train_names = os.listdir(train_dir)
print(train_names)
test_names = os.listdir(train_dir)
print(test_names)

['A', 'G', 'S', 'W', 'N', 'L', 'H', 'X', 'K', 'V', 'P', 'I', 'T', 'B', 'C', 'R', 'del', 'J', 'F', 'D', 'Z', 'space', 'E', 'Q', 'M', 'U', 'Y', 'nothing', 'O']
['A', 'G', 'S', 'W', 'N', 'L', 'H', 'X', 'K', 'V', 'P', 'I', 'T', 'B', 'C', 'R', 'del', 'J', 'F', 'D', 'Z', 'space', 'E', 'Q', 'M', 'U', 'Y', 'nothing', 'O']


**Build the Model**

In [0]:
import tensorflow as tf

In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 128 neuron hidden layer
    tf.keras.layers.Dense(128, activation = tf.nn.relu),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(29, activation = tf.nn.softmax)
])

**Inspect the Layers**

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 33856)             0

**Set the optimizer** <br>
Use adam for it's adaptive learning rate

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

**Define the ImageGenerator**

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(200, 200),
        batch_size=100,
        # Since we use categorical_crossentropy loss, we need sparse labels
        class_mode='sparse')


Found 87000 images belonging to 29 classes.


**Train**

In [0]:
model.fit(train_generator, 
          steps_per_epoch = 870, #we have 87,000 images, batch size of 100
          epochs=10)

Epoch 1/10
870/870 [==============================] - 166s 190ms/step - loss: 0.0436 - accuracy: 0.9863
Epoch 2/10
870/870 [==============================] - 169s 194ms/step - loss: 0.0312 - accuracy: 0.9903
Epoch 3/10
870/870 [==============================] - 171s 196ms/step - loss: 0.0260 - accuracy: 0.9919
Epoch 4/10
294/870 [=========>....................] - ETA: 1:53 - loss: 0.0120 - accuracy: 0.9962

***Debugging***

In [14]:
#!ls '/tmp/'
#!ls '/tmp/asl-alphabet/'
!ls '/tmp/asl_alphabet_train/'
!ls '/tmp/asl_alphabet_train/asl_alphabet_train'
#!ls '/tmp/asl-alphabet/asl_alphabet_train/asl_alphabet_test'

asl_alphabet_train
A  C  del  F  H  J  L  N	O  Q  S      T	V  X  Z
B  D  E    G  I  K  M  nothing	P  R  space  U	W  Y
